In [1]:
import pandas as pd
import re
from textblob import TextBlob
from textblob import Word
from nltk.corpus import stopwords
import deep_translator
from deep_translator import GoogleTranslator
from os.path import exists

nrows = 1000000
data_filename = "tweets.csv"
preprocessed_data_filename = "preprocessed_tweets.csv"
cols = ["timestamp", "text"]
translator = GoogleTranslator(source="auto", target="en")
timestamp_text_dict = {"timestamp": [], "text": []}

if not exists(preprocessed_data_filename):
    df = pd.read_csv(data_filename, sep=';', nrows=nrows, usecols=cols, lineterminator="\r")
    for index, row in df.iterrows():
        if type(row["text"]) == str and row["text"].isascii():
            # Regular expression from https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/
            cleaned = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", row["text"]).split())
            textblob = TextBlob(cleaned)
            stop_words_removed = []
            for word in textblob.words:
                if word not in stopwords.words():
                    lemmatized_word = Word(word).lemmatize()
                    stop_words_removed.append(lemmatized_word)
            timestamp_text_dict["text"].append(" ".join(stop_words_removed))
            timestamp_text_dict["timestamp"].append(row["timestamp"])

df = pd.DataFrame.from_dict(timestamp_text_dict)
df.to_csv(preprocessed_data_filename, index=False)